In [ ]:
# 初始化流動性資料
liquidity_pairs = {
    ("tokenA", "tokenB"): (17, 10),
    ("tokenA", "tokenC"): (11, 7),
    ("tokenA", "tokenD"): (15, 9),
    ("tokenA", "tokenE"): (21, 5),
    ("tokenB", "tokenC"): (36, 4),
    ("tokenB", "tokenD"): (13, 6),
    ("tokenB", "tokenE"): (25, 3),
    ("tokenC", "tokenD"): (30, 12),
    ("tokenC", "tokenE"): (10, 8),
    ("tokenD", "tokenE"): (60, 25),
}
# 添加逆向配對的流動性
enhanced_liquidity = {}
for pair, reserves in liquidity_pairs.items():
    enhanced_liquidity[pair] = reserves
    enhanced_liquidity[(pair[1], pair[0])] = (reserves[1], reserves[0])

# 計算交換後的數量
def calculate_amount_out(injected_amount, input_reserve, output_reserve):
    adjusted_amount = injected_amount * 997
    output_numerator = adjusted_amount * output_reserve
    output_denominator = input_reserve * 1000 + adjusted_amount
    return output_numerator / output_denominator

# 執行交換操作
def execute_swap(input_token, output_token, injected_amount, liquidity_data):
    output_amount = calculate_amount_out(injected_amount, liquidity_data[(input_token, output_token)][0], liquidity_data[(input_token, output_token)][1])
    liquidity_data[(input_token, output_token)] = (liquidity_data[(input_token, output_token)][0] + injected_amount, liquidity_data[(input_token, output_token)][1] - output_amount)
    liquidity_data[(output_token, input_token)] = (liquidity_data[(input_token, output_token)][1], liquidity_data[(input_token, output_token)][0])

# 尋找套利機會
def find_arbitrage_opportunities(current_token, amount, transaction_path, visited_pairs, liquidity_data):
    if current_token == "tokenB" and len(transaction_path) > 1:
        return amount, transaction_path
    temp_liquidity = liquidity_data.copy()
    max_amount = 0
    optimal_path = []
    for (token1, token2), _ in temp_liquidity.items():
        if token1 == current_token and (token1, token2) not in visited_pairs:
            visited_pairs.add((token1, token2))
            next_amount = calculate_amount_out(amount, temp_liquidity[(token1, token2)][0], temp_liquidity[(token1, token2)][1])
            execute_swap(token1, token2, amount, temp_liquidity)
            if next_amount > 0:
                resulting_amount, path = find_arbitrage_opportunities(token2, next_amount, transaction_path + [token2], visited_pairs, temp_liquidity)
                if resulting_amount > max_amount:
                    max_amount = resulting_amount
                    optimal_path = path
            visited_pairs.remove((token1, token2))
    return max_amount, optimal_path


In [ ]:
bp = find_arbitrage_opportunities("tokenB", 5, ["tokenB"], set(), newLq)[1]


In [ ]:
for i in range(len(bp) - 1):
    # address[] memory route = new address[](2);
    #     route[0] = address(tokenB);
    #     route[1] = address(tokenD);
    print(f"route[0] = address({bp[i]});")
    print(f"route[1] = address({bp[i+1]});")
    print(f"console.log({bp[i]}.balanceOf(arbitrager));")
    print(f"router.swapExactTokensForTokens({bp[i]}.balanceOf(arbitrager), 0, route, arbitrager, block.timestamp);")
